# Introduction

In [ ]:
# Aim: To get word-associations (rules) using a database of words
# words occuring together are more important, and may suggest some kind of association between them
# eg. A is type-of B, A is subset/subtype of B etc.

# ARM resources terminology
# Database(D) - Consists of several transactions (T1, T2, ...)
# Items (I) - Items present in market basket (i1, i2, ...)
# Every transaction consists of a subset of Items

# Our terminology
# Database (D) - Consists of several tables (sets the universe) which specify columns to look at (C1, C2, ...)
# Items (I) - Words in the columns, with or without filtering (removing stop-words, lemmatization etc) (w1, w2, ...)
# Every column consists of a (small) subset of Items

In [ ]:
# Plan
# Let user specify set of tables to consider as database
# Use columns inside that to generate word associations
# Plot the results of Support, Confidence of rule etc..

# Imports

In [ ]:
!pip install pycaret
# !pip install pydot

In [ ]:
import os 
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
import traceback
from bs4 import BeautifulSoup, SoupStrainer
import sys
import re
import pandas as pd
import numpy as np
import networkx as nx
import nltk
from nltk.corpus import stopwords, words as nltk_corpus_words
from nltk.stem import WordNetLemmatizer
# from english_words import english_words_set
import time
import json
import wordcloud
import functools
from pycaret import arules
from pycaret.utils import enable_colab
from networkx.drawing.nx_agraph import graphviz_layout
import random
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('words')
# nltk.download('averaged_perceptron_tagger')

In [ ]:
from IPython.display import HTML
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# enable_colab()

# Set up

In [ ]:
# Get table_columns_info from saved json file (generated by Intra_Entity_Analysis.ipynb)
# Generate a dict {column_name : words} (as transaction database)
# Generate a dict {table_name : columns} (to know which columns(transactions) to take for particular table - helps in setting up database)
table_columns_info_file = "/content/drive/MyDrive/Process_Mining_SOP/Sharanya/OutcomeFolder/table_column_info.json"

with open(table_columns_info_file) as infile:
  table_columns_info_json = json.load(infile)

In [ ]:
column_words_dict = {}
table_columns_dict = {}

for filename, column_data in table_columns_info_json.items():
  if filename not in table_columns_dict:
    table_columns_dict[filename] = list(column_data.keys())
  else:
    print(f"Table {filename} found again")

  for column_name, column_info in column_data.items():
    if column_name not in column_words_dict:
      column_words_dict[column_name] = column_info['words']
    else:
      continue
      print(f"Column {column_name} encountered before")


In [ ]:
len(table_columns_dict)

879

In [ ]:
len(table_columns_info_json)

879

# Options

In [ ]:
pd.set_option('display.max_rows', None)

# ARM

In [ ]:
# Take in a list of tables to set up the database
def get_database(database_table_list):
  global table_columns_dict, column_words_dict

  database = {'column_name':[], 'word':[], 'from_table':[]}
  columns_taken = set()

  for table in database_table_list:
    if table not in table_columns_dict:
      print(f"Table {table} not found in table_columns_dict")
      continue
    for column in table_columns_dict[table]:
      if column in columns_taken:
        continue
      else:
        columns_taken.add(column)
        for word in column_words_dict[column]:
          database['column_name'].append(column)
          database['word'].append(word)
          database['from_table'].append(table)
  
  return database

In [ ]:
random_table_list = list(table_columns_dict.keys())
random.shuffle(random_table_list)
random_table_list_selection = random_table_list[0::10]
len(random_table_list_selection)

88

In [ ]:
def find_columns(word_list):
  global database_df

  word_count_per_col = database_df[database_df['word'].isin(word_list)].groupby(by='column_name').agg({'word':'count', 'from_table':'first'}).reset_index()
  columns_satisfying = word_count_per_col[word_count_per_col['word'] == len(word_list)][['column_name', 'from_table']].values

  return columns_satisfying.tolist()

In [ ]:
def set_to_string(given_set):
  # print(given_set)
  # if len(given_set) > 1:
  #   return None
  # else:
  #   return list(given_set)[0]
  return ' '.join(list(given_set))

In [ ]:
def convert_set_to_string(df_series):
  new_arr = pd.Series(list(map(set_to_string, df_series)))
  return new_arr

In [ ]:
def reduce_consequents(series):
  return functools.reduce(lambda x, y: x+','+y, series)

In [ ]:
def get_columns_tables(row):
  # print(f"row is\n{row}")
  all_words = row['antecedents'].split()
  all_words.extend(row['consequents'].split())

  cols_tables = '\n'.join([col_table[0]+" : "+col_table[1] for col_table in find_columns(all_words)])
  # print(f"returning {tuple([len(cols_tables), cols_tables])}")
  return cols_tables

In [ ]:
num_of_tables_per_run = 100
num_of_runs = int(np.ceil(len(random_table_list)/num_of_tables_per_run))

for i in range(num_of_runs):
  print(f"On batch {i}")
  current_table_list = random_table_list[i*num_of_tables_per_run:(i+1)*num_of_tables_per_run]
  database = get_database(list(current_table_list))
  database_df = pd.DataFrame.from_dict(database)
  print(f"Generated databse")
  setup_result = arules.setup(data=database_df,
                            transaction_id= 'column_name',
                            item_id = 'word')
  model = arules.create_model(threshold=0, min_support=0.1/(len(database_df['column_name'].unique())), max_len=2)
  print(f"Model shape is {model.shape}")
  model['consequents'] = convert_set_to_string(model['consequents'])
  model['antecedents'] = convert_set_to_string(model['antecedents']) 
  model.sort_values('confidence', ascending=False, inplace=True)
  print(f"Generated final model")
  model[['antecedents', 'consequents']].to_csv('/content/drive/MyDrive/Process_Mining_SOP/'+'random_'+str(num_of_tables_per_run)+'_tables_batch_'+str(i)+'.csv', header=False, index=False)
  print(f"Model saved")
  print()


On batch 0
Generated databse


Description,Value
session_id,6594
# Transactions,784
# Items,488
Ignore Items,None


Model shape is (1642, 9)
Generated final model
Model saved

On batch 1
Generated databse


Description,Value
session_id,1455
# Transactions,818
# Items,514
Ignore Items,None


Model shape is (1682, 9)
Generated final model
Model saved

On batch 2
Generated databse


...

Model shape is (2266, 9)
Generated final model
Model saved

On batch 7
Generated databse


Description,Value
session_id,5745
# Transactions,1000
# Items,612
Ignore Items,None


Model shape is (2308, 9)
Generated final model
Model saved

On batch 8
Generated databse


Description,Value
session_id,2164
# Transactions,795
# Items,457
Ignore Items,None


Model shape is (1414, 9)
Generated final model
Model saved



# Running ARM on chains

In [ ]:
# for i in range(num_of_runs):
#   print(f"Moving to file {i+1} of {num_of_runs}")
#   with open('/content/drive/MyDrive/Process_Mining_SOP/'+'chains_random_'+str(num_of_tables_per_run)+'_tables_batch_'+str(i)+'.csv', 'r') as f:
#     for line in f:
#       global_chain_num += 1
#       words = line.strip(', ').split(',')[:-2]
#       if len(words) == 1:
#         continue
#       for word in words:
#         chain_database['chain_id'].append(global_chain_num)
#         chain_database['word'].append(word)
      


Moving to file 1 of 2
Moving to file 2 of 2


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
# chain_database = {'chain_id':[], 'word':[]}
all_chains_database = []
global_chain_num = 0
num_of_runs=9
num_of_tables_per_run=100

In [ ]:
for i in range(3, 6):
  print(f"Moving to file {i+1} of {num_of_runs}")
  with open('/content/drive/MyDrive/Process_Mining_SOP/'+'chains_random_'+str(num_of_tables_per_run)+'_tables_batch_'+str(i)+'.csv', 'r') as f:
    for line in f:
      global_chain_num += 1
      words = line.strip(', ').split(',')[:-2]
      all_chains_database.append(words)
      

Moving to file 4 of 9
Moving to file 5 of 9
Moving to file 6 of 9


In [ ]:
te = TransactionEncoder()
te_ary = te.fit(all_chains_database).transform(all_chains_database)
chains_df = pd.DataFrame(te_ary, columns=te.columns_)
len(chains_df)

34113

In [ ]:
frequent_itemsets = apriori(chains_df, min_support=5/len(chains_df), use_colnames=True, low_memory=True, verbose=2)
# frequent_itemsets

Processing 13 combinations | Sampling itemset size 12


In [ ]:
frequent_itemsets.sort_values(by='support')

,support,itemsets,itemsets_formatted,items_count,support_num
8148,0.000223,"(attribute, name, lot)",attribute name lot,3,5.0
64357,0.000223,"(strategy, pick, auth, time, call)",strategy pick auth time call,5,5.0
16209,0.000223,"(time, level, freeze)",time level freeze,3,5.0
34540,0.000223,"(transaction, drop, cond, extended)",transaction drop cond extended,4,5.0
29083,0.000223,"(date, status, audit, maintained)",date status audit maintained,4,5.0
...,...,...,...,...,...
482,0.144407,(task),task,1,3234.0
112,0.162090,(cost),cost,1,3630.0
306,0.165483,(num),num,1,3706.0
246,0.196919,(item),item,1,4410.0


In [ ]:
frequent_itemsets['itemsets_formatted'] = convert_set_to_string(frequent_itemsets['itemsets'])
frequent_itemsets['items_count'] = series_count_items(frequent_itemsets['itemsets_formatted'])
frequent_itemsets['support_num'] = frequent_itemsets['support']*len(all_chains_database)

In [ ]:
frequent_itemsets[['support', 'support_num', 'itemsets_formatted', 'items_count']].to_csv('/content/drive/MyDrive/Process_Mining_SOP/'+'frequent_itemsets_batch_01.csv')
print(f"Frequent itemsets saved")
print()

Frequent itemsets saved



In [ ]:
chain_database_df = pd.DataFrame.from_dict(chain_database)

In [ ]:
chain_database_df[chain_database_df['chain_id'] == 5208]

,chain_id,word
29347,5208,extended
29348,5208,drop
29349,5208,cond
29350,5208,program
29351,5208,price
29352,5208,set
29353,5208,value
29354,5208,min
29355,5208,unit
29356,5208,num


In [ ]:
chain_database_df.groupby(by='chain_id').agg({'word':'count'}).sort_values('word')

,word
chain_id,
22394,2
14991,2
21443,2
14995,2
21445,2
...,...
12258,13
12248,13
5208,13


In [ ]:
setup_result = arules.setup(data=chain_database_df,
                            transaction_id= 'chain_id',
                            item_id = 'word')

Description,Value
session_id,5614
# Transactions,21809
# Items,673
Ignore Items,None


In [ ]:
# 100000/(len(chain_database_df['chain_id'].unique()))
start_time = time.time()
model = arules.create_model(threshold=0, min_support=0.00005, low_memory=True)
end_time = time.time()

print(f"Model made has shape {model.shape}")
print(f"Time taken = {end_time-start_time}")

In [ ]:
model.shape

(58, 9)

In [ ]:
model

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(tran, set, warn)",(locn),0.0194,0.0944,0.0194,1.0000,10.5972,0.0176,inf
1,"(unit, item, set)",(num),0.0102,0.1699,0.0102,1.0000,5.8848,0.0084,inf
2,"(level, unit, value)",(num),0.0101,0.1699,0.0101,1.0000,5.8848,0.0084,inf
3,"(level, num, value)",(unit),0.0101,0.1073,0.0101,1.0000,9.3161,0.0090,inf
4,"(min, tran, num, set)","(unit, value)",0.0110,0.0656,0.0110,1.0000,15.2510,0.0103,inf
...,...,...,...,...,...,...,...,...,...
6557,(pick),"(reversal, strategy)",0.1226,0.0127,0.0124,0.1010,7.9786,0.0108,1.0982
6558,(node),"(access, item)",0.1453,0.0146,0.0146,0.1004,6.8842,0.0125,1.0954
6559,(node),"(factor, access)",0.1453,0.0146,0.0146,0.1004,6.8842,0.0125,1.0954
6560,(tran),"(pick, zone)",0.2300,0.0312,0.0230,0.1001,3.2051,0.0158,1.0765


In [ ]:
model['consequents'] = convert_set_to_string(model['consequents'])
model['antecedents'] = convert_set_to_string(model['antecedents'])

In [ ]:
def count_items(s):
  return len(s.split())

In [ ]:
def series_count_items(df_series):
  new_arr = pd.Series(list(map(count_items, df_series)))
  return new_arr

In [ ]:
model['count_items'] = series_count_items(model['antecedents'])
model['count_items'] += series_count_items(model['consequents'])

In [ ]:
model[model['count_items'] == 6]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,count_items
4,min tran num set,unit value,0.0110,0.0656,0.0110,1.0000,15.2510,0.0103,inf,6
24,locn unit set value num,tran,0.0176,0.2300,0.0176,1.0000,4.3479,0.0136,inf,6
26,min tran num value,unit set,0.0110,0.0441,0.0110,1.0000,22.6705,0.0105,inf,6
29,min tran num unit,value set,0.0110,0.0713,0.0110,1.0000,14.0341,0.0102,inf,6
78,min tran num,unit set value,0.0110,0.0441,0.0110,1.0000,22.6705,0.0105,inf,6
...,...,...,...,...,...,...,...,...,...,...
6520,unit,value set zone tran num,0.1073,0.0110,0.0110,0.1025,9.3161,0.0098,1.1020,6
6521,unit,value set tran num task,0.1073,0.0110,0.0110,0.1025,9.3161,0.0098,1.1020,6
6528,value,unit set zone tran num,0.1078,0.0110,0.0110,0.1021,9.2765,0.0098,1.1014,6
6539,value,unit set tran num task,0.1078,0.0110,0.0110,0.1021,9.2765,0.0098,1.1014,6
